# How-to: use Anisotropic Repetition Codes (or ARCs)

Anisotropic repetition codes are quantum implementations of the repetition code. Standard repetition codes store a logical bit value on a set of qubits (the 'code qubits') by repeating the value to be stored. The two standard versions are the z and x basis encodings, also known as the bit and phase encodings. For the z basis, the bit state `0` corresponds to the state $|0\rangle^{\otimes d}$ for $d$ code qubits, and `1` corresponds to $|1\rangle^{\otimes d}$. For the x basis encoding, `0` and `1` correspond to $|+\rangle^{\otimes d}$ and $|-\rangle^{\otimes d}$ respectively. These encodings are sensitive to and protect against bit flip and phase flip noise respectively, but neither is senstive to both.

Anisotropic repetition codes differ in that they use different bases on different qubits. This means that any run of the code is sensitive to all kinds of error, even though each individual qubit is sensitive only to one. For example, a `basis='zx'` repetition code alternates between the z and x bases. For example, using the code qubit state $|0,+,0,+,\ldots\rangle$ to store `0` or $|1,-,1,-,\ldots\rangle$ to store `1`.

Anisotropic repetition codes are intended as a way of benchmarking near-term quantum devices, and their progress towards fault-tolerance.

## Making an ARC

ARCs are all based around the `ArcCircuit` class. So our first job is to import it.

In [ ]:
from qiskit_qec.circuits.repetition_code import ArcCircuit

ARCs are basically repetition codes defined on a general graph. We therefore begin by specifying what job the qubits will play in the code, and where the two-qubit parity measurements are implemented. This is all based on the connectivity of the qubits.

For example, suppose we have 5 qubits in a line

    0 -- 1 -- 2 -- 3 -- 4

Here entangling gates can be directly implement only between neighbours. Two qubit parity measurements could therefore be made on qubits `0` and `2` (using `1` as an aux), and on qubits `2` and `4` (using `3` as an aux). We will express this using tuples `(c0, a, c1)`, where `c0` and `c1` are the code qubits on which the parity measurement is made, and `a` is the aux. For example, we have `(0,1,2)` and `(2,3,4)` in this case. All such tuples are then compiled in a list.

In [ ]:
links = [
    (0,1,2),
    (2,3,4)
]

The only remaining information required to specify an ARC is the number of syndrome measurement rounds to be included in the circuit. For example, let's us two.

In [ ]:
T = 2

Now we can create an ARC on these five qubits, with these syndrome measurements run for this number of rounds.

In [ ]:
code = ArcCircuit(links, T)

During initialization, the ARC tries to work out a good way to schedule entangling gates within a round. It doesn't always do a good job. You can see what it has done with the following.

In [ ]:
code.schedule

If this has yielded something like

    [
        [[0, 1]],
        [[2, 1]],
        [[2, 3]],
        [[4, 3]]
    ]

it hasn't done a very good job. This represents first completely measuring the first link, and then doing the second. Specifically, it does the two-qubit gate between `0` and `1`, then that between `2` and `1`, then `2` and `3`, and finally `4` and `3`.

A more efficient method would be to do non-overlapping gates from both measurements in parallel, and then do the same again for the remaining gates. For example

In [ ]:
schedule = [
        [[0, 1], [2, 3]],
        [[2, 1], [4, 3]]
    ]

This represents doing the `0` and `1` gate in parallel with the `2` and `3` gate, and then the same with the `2` and `1` gate and `4` and `3` gate.

In cases like this when we know a better schedule, we can give it to the code upon initialization. Then it will use that instead of calculating its own.

In [ ]:
code = ArcCircuit(links, T, schedule=schedule)

code.schedule

Another thing that the code does on initialization is to bicolor the code qubits. Each qubit is assigned a 'color' of `0` or `1`, ideally such that each qubit is differently colored than those it shares links with. The ARC finds a bicoloring that does a good (but not perfect) job.


In [ ]:
code.color

For example, the above may yield something like

    {0: 1, 2: 0, 4: 0}

Here the qubits `0` and `2` are differently colored, as we want from a pair that shares a link. However, qubits `2` and `4` also share a link but have the same color. This is not due to the fact that no such bicoloring is possible, since it could be done with

In [ ]:
color = {0: 0, 2: 1, 4: 0}

So, again, when we know a optimal strategy, we can tell the ARC about it upon initialization.

In [ ]:
code = ArcCircuit(links, T, schedule=schedule, color=color)

code.color

The bicoloring is used to determine how exactly the code alternates between the two given bases. For `basis='zx'`, qubits of color `0` are stored with the z basis and `1` with the x basis.

In this anisotropic case, each code qubit would still only detect a limited set of errors (bit flip errors for the z-basis encoding, and phase flip errors for the x-basis). However, the nature of the bicoloring ensures that there is always something nearby to catch large-scale forms of any error.

To ensure that all kinds of errors can be detected on all qubits, we can run a separate circuit but with the opposite bases. Of course, this doesn't work on the shot-by-shot basis for which errors should be detected in standard quantum error correction. However, it will be sufficient in benchmarks where we simply measure the prevalence of different error types.

The desired bases are supplied as follows.

In [ ]:
basis = 'zx'

code = ArcCircuit(links, T, schedule=schedule, color=color, basis=basis)

The ARC then creates two circuits for this form of anistropy. One with the bases this way round, and the other with the opposite. Both correspond to a stored bit state of `'0'`.

In [ ]:
code.circuit

By default, ARCs are built using `basis='xy'`.

## Running an ARC

These circuits can be run on simulators in the normal way.

In [ ]:
from qiskit import Aer

backend = Aer.get_backend('aer_simulator')

backend.run(code.circuit[code.basis]).result().get_counts()

The output strings are in the same form as for `RepetitionCodeCircuit`, with the final code qubit readouts and each round of syndrome measurement all separated.

Outcomes can be interpreted by determining the corresponding nodes of the syndrome graph. The trivial case of no errors (as above) gives us no nodes.

In [ ]:
code.string2nodes('000 00 00')

For an example of a result with an error, let's consider `'010 11 00'`. This has `00` for the syndrome in the first round (implying no error), then `11` for the second round (both detect an error) and a final code qubit readout of `010` in which we see that the middle code qubit is flipped relative to the others. This implies that the middle qubit flipped between the first and second rounds. The corresponding nodes are

In [ ]:
code.string2nodes('010 11 00')

To run on a real device, we need to be aware of that device's needs. For example, let's use `'ibmq_jakarta'` (or at least pretend to).

In [ ]:
from qiskit.providers.fake_provider import FakeJakarta

backend = FakeJakarta()

This has the coupling map

    0 -- 1 -- 2
         |
         3
         |
    4 -- 5 -- 6

So even to do the same 5-qubit code as before, we need to rewrite the links in terms of these 7 qubits.

In [ ]:
links = [
    (2,1,3),
    (3,5,6)
]

schedule = [
        [[2, 1], [3, 5]],
        [[3, 1], [6, 5]]
    ]
    
color = {2: 0, 3: 1, 6: 0}

code = ArcCircuit(links, T, schedule=schedule, color=color, basis=basis)

The ARC contains all you need to transpile the circuits to this backend, and created scheduled circuits. This includes inserting dynamcal decoupling. By default, it inserts a pair of `x` gates into any delays on the code qubits only.

In [ ]:
circuit = code.transpile(backend)

Let's take a look at them in all their glory!

In [ ]:
from qiskit.visualization.timeline import draw

draw(circuit[code.basis])

## `[[2,0,2]]`s

Another significant difference between standard repetition codes and ARCs is the presence of `[[2,0,2]]` codes in the latter. This is the sequential testing of each link by alternating between the standard basis for that link (`zx` for example) and its opposite (`xz` in this case). By doing so we can simultaneously detect all kinds of errors on that particular link. This does come at the cost of temporarily disrupting the code around it, which is why the links are done one by one.

The ARCs above don't actually include the `[[2,0,2]]`s, as we can see below.

In [ ]:
code.run_202

This is because the code is not big enough. Just as the `RepetitionCodeCircuit`s use the code qubits on the ends of the line as logical readouts, ARCs use any code qubit that is part of only one link. If no such code qubit exists, it just uses the first.

For the current code (defined in the last section) there are ends at code qubits `0` and `6`.

In [ ]:
code.z_logicals

The `[[2,0,2]]`s will skip any links that include code qubit readouts. For the current code, that means it skips all links.

So let's make a bigger code (not designed for `'ibmq_jakarta'` this time) with at least one link to do a `[[2,0,2]]` on.

In [ ]:
links = [
    (0,1,2),
    (2,3,4),
    (4,5,6)
]

Another requirement is for `T` to be large enough: it must be at least 5 times the number of links on which `[[2,0,2]]`s wil be run. So in this case, with one `[[2,0,2]]` link, we need at least `T=5`. However, you can always be sure to have enough with

In [ ]:
T = 5*len(links)

The code will then run through all the links (in the order of `links`) spending 5 rounds on each `[[2,0,2]]`.

Here is a code for which `[[2,0,2]]`s are present.

In [ ]:
code = ArcCircuit(links, T, barriers=True, basis='xz', color = {0: 0, 2: 1, 4: 0, 6: 1})

code.run_202

The code deformation introduced by the `[[2,0,2]]`s means that we will get some degree of randomness in the results, even when no errors are present.

In [ ]:
backend = Aer.get_backend('aer_simulator')

counts = backend.run(code.circuit[code.basis]).result().get_counts()
counts

But `string2nodes` knows how to account for these, such that they don't correspond to any nodes.

In [ ]:
for string in counts:
    print(code.string2nodes(string))

## Decoding

In [ ]:
from qiskit_qec.decoders.hdrg_decoders import ClusteringDecoder

To do decoding of an ARC, we need to choose a decoder. The choice is currently simple, since only one decoder is compatible: the `ClusteringDecoder`. We begin by creating a decoder object.

In [ ]:
decoder = ClusteringDecoder(code)

Then we need something to decode. Specifically, a string that forms a valid output for the code. Let's start with the trivial case, that occurs when no errors are present.

In [ ]:
string = '0000 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000'

The deocder will tell us what it thinks the outcomes of the z logicals are, given this string. For this code these logical readouts correspond to the following qubits.

In [ ]:
code.z_logicals

Since there are two, the decoder will return a list of two values: one for each. Now let's run the decoder.

In [ ]:
decoder.process(string)

This output says that the logical bit was read out as `0` on both qubits used for the logical readout. However, consider the case where an error occurred on all code qubits directly before final readout.

In [ ]:
string = '1111 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000'

decoder.process(string)

In this case we see that the decoder assumes that the logical bit was stored as `1`, which is a logical error.

However, if the errors occurred only on the two qubits used for logical readout

In [ ]:
string = '1001 000 000 000 000 000 000 000 000 000 000 000 000 000 000 000'

the decoder would correctly deduce that these were isolated errors and correct for them.

In [ ]:
decoder.process(string)

To see how it works, let's look at a code with more code qubits.

In [ ]:
d = 9
T = 5

links = [(2*j, 2*j+1, 2*(j+1)) for j in range(d-1)]
code = ArcCircuit(links, T)
decoder = ClusteringDecoder(code)


The following string corresponds to an error on the middle code qubit just before the final syndrome measurement round, as well as a measurement error on one of the qubits in the first round.

In [ ]:
string = '000010000 00011000 00000000 00000000 00000000 00000010'

We can then see what nodes this corresponds to in the decoding graph.

In [ ]:
nodes = code.string2nodes(string)
nodes

These nodes can be more compactly described by their indices.

In [ ]:
[decoder.decoding_graph.graph.nodes().index(node) for node in nodes]

The decoder splits the set of nodes into clusters, each of which is likely to have been caused by an independent set of errors.

In [ ]:
decoder.cluster(nodes)

Here nodes 38 and 39 (those for the initial measurement error) are one cluster (numbered 2) and nodes 24 and 30 (for the code qubit error) are in another. The decoder has therefore deduced that they were two separate errors. Whether or not 'boundary' nodes (those associated with logicals) are part of these clusters determines the correction applied.

## Understanding the outputs

When we look at the output of an ARC circuit, for example

    '1111 000 000 000 000 010 000 000 000 001 000 000 000 000 000 100'

each of the different blocks corresponds to a different register. The registers in the circuit are as follows.

In [ ]:
code.circuit[code.base].cregs

Specifically, the blocks shown from right to left in the output string correspond to the registers listed from top to bottom in the above list of registers. So the `100` from the example string is for register `ClassicalRegister(3,'round_0_link_bit')`, the `001` is `ClassicalRegister(3,'round_6_link_bit')` and `11111` is `ClassicalRegister(4,'code_bit')`.

The link bit registers contain the outcomes for measurements of the auxiliary bits at the center of each link. The bits are indexed from right to left. The index corresponding to the result for each auxiliary bit corresponding to each index can be found in `code.link_index`.

In [ ]:
for a,j in code.link_index.items():
    print('The outcome for auxiliary qubit',a,'is found at index',j,'')

The code bit register corresponds to the final measurement of all code qubits. The index corresponding to the result for each auxiliary bit corresponding to each index can be found in `code.code_index`.

In [ ]:
for a,j in code.code_index.items():
    print('The outcome for code qubit',a,'is found at index',j,'')

As an example of using this information, we'll now look at how to generate the kind of output strings that could be expected when errors occur.

To make things easy, we'll do that for a version of the code that doesn't have `[[2,0,2]]`s.

In [ ]:
code = ArcCircuit(links, T, run_202=False)

First let's make a string without errors. For this we just need to know the number of code qubits, the number of link qubits and the number of rounds.

In [ ]:
def generate_clean_string(code, logical='0'):
    '''
    Creates an output string for the given code in which no errors
    are detected.
    '''
    string = logical*len(code.code_index)
    for _ in range(code.T):
        string += ' ' + '0'*len(code.link_index)

    return string

generate_clean_string(code)

The following function then takes an existing string and adds a single error to it, on a given qubit and at a given measurement round.

In [ ]:
def add_error(string, q, t):
    '''
    Adds an error to the given string for qubit q at round t.
    '''

    # make sure the code has the properties that we assume for the following
    assert code.run_202==False
    assert code._resets==True

    # turn string into an array where elements can be accessed by their
    # index and round number as string_array[t][j]
    string_array = []
    for block in string.split(' ')[::-1]:
        string_array.append(list(block)[::-1])

    # if the error is on an auxiliary qubit, it causes the value for that round to flip
    if q in code.link_index:
        j = code.link_index[q]
        string_array[t][j] = str((int(string_array[t][j]) + 1)%2)

    # 
    if q in code.code_index:
        # flip its value in the final readout
        j = code.code_index[q]
        string_array[-1][j] = str((int(string_array[-1][j]) + 1)%2)
        # find the auxiliaries for neighbouring links
        neighbors = []
        for link in links:
            if q in link:
                neighbors.append(link[1])
        # flip them all from the round when the error happend until the end
        for qq in neighbors:
            for tt in range(t,code.T):
                j = code.link_index[qq]
                string_array[tt][j] = str((int(string_array[tt][j]) + 1)%2)
        
    # turn the array back into a string
    new_string = ''
    for block in string_array[::-1]:
        new_string += ''.join(block[::-1]) + ' '
    new_string = new_string[0:-1]

    return new_string

string = add_error(generate_clean_string(code), 1, 4)

For example, an error on the qubit 3 (an auxiliary) at round 5.

In [ ]:
string = add_error(generate_clean_string(code), 3, 5)
string

This corresponds to the following nodes.

In [ ]:
code.string2nodes(string)

Another example is an error on qubit 2 (a code qubit) prior to round 9.

In [ ]:
string = add_error(generate_clean_string(code), 2, 9)
print(string)
code.string2nodes(string)

We could also combine the two.

In [ ]:
string = generate_clean_string(code)
string = add_error(string, 3, 5)
string = add_error(string, 2, 9)
print(string)
code.string2nodes(string)